In [1]:
#r "C:\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [1]:
WorkflowMgm.Init("SIP_SingleNodePerformance_0");

In [1]:
using System.IO;
using BoSSS.Application.SipPoisson;

In [1]:
string dateString = "2020-10-23";
string File       = "SolverRuns--" + dateString + ".json";
string FilePath   = Path.Combine(CurrentDocDir, File);
var SessTab       = TableExtensions.LoadFromFile(FilePath);

In [1]:
//SessTab.Print();

In [1]:
LinearSolverCode[] Solvers = new LinearSolverCode[] {
LinearSolverCode.exp_Kcycle_schwarz,
LinearSolverCode.exp_gmres_levelpmg
};

In [1]:
// ====================
// Evaluation, plotting
// ====================

In [1]:
SessTab.GetColumnNames();

In [1]:
Func<IDictionary<string,object>,bool> solverselection = delegate(IDictionary<string,object> row){
bool selectsolver = false;
foreach(var solver in Solvers)
    selectsolver |= (Convert.ToInt32(row["LinearSolver.SolverCode"]) == (int)solver);
return selectsolver;
}

In [1]:
var SubTab_1 = SessTab.ExtractRows(delegate(int i, IDictionary<string,object> row) {
    return (  solverselection(row) && (Convert.ToInt32(row["Conv"]) != 0));
    });

In [1]:
var SubTab = SubTab_1.ExtractColumns("DGdegree:T", "Grid:NoOfCells", "LinearSolver.SolverCode", "DOFs", 
              "minSolRunT", "maxSolRunT", "UsedMGDepth",
              "MatrixAssembly", "Aggregation_basis_init_time", "Solver_Init_time", "Solver_Run_time");

In [1]:
SubTab.Print();

In [1]:
//extract all Degrees and put them into array
int rowcount       = SubTab.Rows.Count;
int selectedcolumn = 0;
int[] tmp= new int[20];

int count = 0;
for(int r=0;r<rowcount;r++){
System.Data.DataRow row = (System.Data.DataRow)SubTab.Rows[r];
int bla                 = Convert.ToInt32(row[selectedcolumn]);
if (tmp[bla]==0){
  tmp[bla]=1;
  count++;
}
}
int[] DGdegrees = new int[count];
int count2 = 0;
for(int i=0;i<tmp.Length;i++){
   if(tmp[i]==1){
      DGdegrees[count2]=i;
      count2++; 
   }
}
DGdegrees;

In [1]:
//cl.WriteMinimalCompileableExample("C:\\tmp\\mini.tex");

In [1]:
int rows = 3;
int columns = 1;

Plot2Ddata[][,] AllMultiplots = new Plot2Ddata[2][,];
for(int iPlot = 0; iPlot < AllMultiplots.Length; iPlot++) {
Plot2Ddata[,] multiplots = new Plot2Ddata[rows,columns];
AllMultiplots[iPlot] = multiplots;
int pDegree = 0;

for(int iRow = 0; iRow < rows; iRow++) {
int iCol = 0; 
    multiplots[iRow,0] = SubTab.ToPlot(
       delegate (int iSweep, int iTabRow, 
                 IDictionary<string, object> Row, 
                 out string Nmn, 
                 out PlotFormat Fmt,
                 out double xValue, out double yValue) { 
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           // PlotRowSelector:
           // selects, which table row goes to which graph,
           // and the respective color  
           // - - - - - - - - - - - - - - - - - - - - - - - - 
           int k                        = Convert.ToInt32(Row["DGdegree:T"]);
           LinearSolverCode solver_name = (LinearSolverCode) Convert.ToInt32(Row["LinearSolver.SolverCode"]);
          
           
           if(    k != DGdegrees[pDegree] 
               || solver_name != Solvers[iPlot]) {
                // degree does not match -> not in this plot
                Nmn    = null;
                Fmt    = null;
                xValue = 0;
                yValue = 0;
                return;
           }
           
           double mtxAssi  = Convert.ToDouble(Row["MatrixAssembly"]);
           double aggBinit = Convert.ToDouble(Row["Aggregation_basis_init_time"]);
           double slvInit  = Convert.ToDouble(Row["Solver_Init_time"]);
           double slvRun   = Convert.ToDouble(Row["Solver_Run_time"]);
           double dofs     = Convert.ToDouble(Row["DOFs"]);
           
           if(aggBinit > 700)
           Console.Write("aggBinit/slvRun = " + aggBinit + "/" + slvRun);
           
           Fmt           = new PlotFormat();
           Fmt.PointSize = 0.5;
           Fmt.Style     = Styles.LinesPoints;
           Fmt.LineColor = LineColors.Black;
           Fmt.LineWidth = 3;
           
           xValue = dofs;
           
           switch(iSweep) {
               case 0:
               yValue       = slvRun;
               Nmn          = "Slv Iter";
               Fmt.DashType = DashTypes.Dotted;
               Fmt.PointType = PointTypes.OpenCircle;
               break;
               
               case 1:
               yValue        = slvInit;
               Nmn           = "Slv Init";
               Fmt.DashType  = DashTypes.Dashed;
               Fmt.PointType = PointTypes.OpenBox;
               break;
               
               case 2:
               yValue        = aggBinit;
               Nmn           = "Agg Init";
               Fmt.DashType  = DashTypes.DotDashed;
               Fmt.PointType = PointTypes.OpenDiamond;
               break;
               
               case 3:
               yValue        = mtxAssi;
               Nmn           = "Mtx ass";
               Fmt.DashType  = DashTypes.DotDotDashed;
               Fmt.PointType = PointTypes.OpenUpperTriangle;
               break;
               
               default:
               throw new NotImplementedException();
           }
           
           if(aggBinit > 700)
           Console.WriteLine("\t + case: {0} + yValue = {1}", iSweep, yValue);
       },
       4 // 4 sweeps over table
       );
 
    string Title = string.Format("$k = {0}$", DGdegrees[pDegree]);
    multiplots[iRow,0].Ylabel = Title;
    multiplots[iRow,0].LogX = true;
    multiplots[iRow,0].LogY = true;
 
    multiplots[iRow,0].YrangeMin = 1e-2;
    multiplots[iRow,0].YrangeMax = 1e+4;
    
    double[] dof = new[]{1e3,1e6};
    double[] linT = dof.Select(x => x*0.001).ToArray();
    var linP = new Plot2Ddata.XYvalues("linear", dof, linT);
    linP.Format.FromString("- black");
    ArrayTools.AddToArray(linP, ref multiplots[iRow,0].dataGroups);
    multiplots[iRow,iCol].XrangeMin = 1e1;
    multiplots[iRow,iCol].XrangeMax = 1e7;
    //multiplots[iRow,iCol].Y2rangeMin = 1e-3;
    //multiplots[iRow,iCol].Y2rangeMax = 1e+4;
    //multiplots[iRow,iCol].X2rangeMin = 1e2;
    //multiplots[iRow,iCol].X2rangeMax = 1e7;
 
    //spacing around plots
    multiplots[iRow,iCol].ShowLegend = false;
    multiplots[iRow,iCol].tmargin = 0;
    multiplots[iRow,iCol].bmargin = 1;
    multiplots[iRow,iCol].lmargin = 10;
    multiplots[iRow,iCol].rmargin = 5;
    multiplots[iRow,iCol].ShowXtics=false;

    //I am legend ...
    if(iRow == 0){
    multiplots[iRow,iCol].ShowLegend = true;
    multiplots[iRow,iCol].LegendAlignment=new string[]{"i","t","l"};
    multiplots[iRow,iCol].LegendSwap=true;
    }

    //and i am special ...
    if(iRow == rows-1)
        multiplots[iRow,iCol].ShowXtics=true;
    
    pDegree++;                       
}
}
;

In [1]:
// check plots
AllMultiplots[0].PlotNow();

In [1]:
// =========
// manually
// =========
//var CL = AllMultiplots[0].ToGnuplot().PlotCairolatex(xSize:14, ySize:12);
//CL.PlotNow();

In [1]:
//CL.WriteMinimalCompileableExample(
//    Path.Combine(CurrentDocDir, "plots", "MainConstCoeffPoisson_Schwarz.tex"),
//    "ConstCoeffPoisson_Schwarz.tex");

In [1]:
// ================
// Loop it
// ==============
for(int i =0; i< Solvers.Length;i++){
string problem = "ConstCoeffPoisson";
string name    = problem + Solvers[i];
var CL         = AllMultiplots[i].ToGnuplot().PlotCairolatex(xSize:14, ySize:12);
CL.WriteMinimalCompileableExample(
    Path.Combine(CurrentDocDir, "plots", "Main"+name+".tex"),
    name+".tex");
}